In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
video = 'VID_highway.mp4'

cap = cv2.VideoCapture(video)

car_cascade = cv2.CascadeClassifier('cars.xml') # By https://github.com/andrewssobral/vehicle_detection_haarcascades

In [3]:
#ROI
line_roi_y1, line_roi_y2, line_roi_x1, line_roi_x2 = 650, 950, 400, 1900
car_roi_y1, car_roi_y2, car_roi_x1, car_roi_x2 = 400, 800, 700, 1900

In [4]:
# Warp perspective windows dimensions
warp_width, warp_height = 300, 450
warp_perc = np.float32([[1250, 650], [1510, 650], [400, 950], [1900, 950]])
warp_rect = np.float32([[0, 0], [warp_width, 0], [0, warp_height], [warp_width, warp_height]])

In [5]:
# Haar Cascade
def car_haar_cascade():
    if car_cascade is not None:
        scaleFactor = 1.1
        minNeighbors = 6
        minSize = 100
        cars = car_cascade.detectMultiScale(roi_car, scaleFactor, minNeighbors, minSize)
        
        return cars
    else:
        return None

In [6]:
#Gaussian Blur
def blur():
    sigma = 19
    roi_b = cv2.GaussianBlur(roi_line, (sigma, sigma), 0)
    
    return roi_b

In [7]:
#Canny Detection
def canny():
    threshold_low = 70
    threshold_high = 100
    roi_c = cv2.Canny(roi_b, threshold_low, threshold_high)
    
    return roi_c

In [8]:
# Draw ROI area
def draw_roi(roi_c):
    '''# Convert from Gray scale to BGR so that it can cover the original frame
    roi_c = cv2.cvtColor(roi_c, cv2.COLOR_GRAY2BGR)    
    # Replace line roi as canny
    frame[line_roi_y1:line_roi_y2, line_roi_x1:line_roi_x2] = roi_c'''
    
    '''# Lane detection roi
    cv2.rectangle(frame, (line_roi_x1, line_roi_y1), (line_roi_x2 ,line_roi_y2), (0, 255, 0), 5)
    # Car detection roi
    cv2.rectangle(frame, (car_roi_x1, car_roi_y1), (car_roi_x2 ,car_roi_y2), (255, 0, 0), 5)'''
    
    # Show Canny detector region in a new windows
    cv2.imshow('ROI', roi_c)

In [9]:
# Draw Hough Lines
def draw_lines():
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            # Avoid drawing horizontal lines(such as bottom of the car / horizon)
            if abs((y2 - y1) / (x2 - x1)) * 180 / np.pi > 20: 
                cv2.line(frame, (x1+line_roi_x1, y1+line_roi_y1), (x2+line_roi_x1, y2+line_roi_y1), [0, 255, 0], 2)

In [10]:
# Show perspective warp
def perspective_warp():
    ''' # Show perspective warp region
    for a in range(0, 4):
        cv2.circle(frame, (warp_perc[i][0], warp_perc[i][1]), 5, (0, 0, 255), cv2.FILLED)''' 
    
    # Convert perspective matrix
    matrix = cv2.getPerspectiveTransform(warp_perc, warp_rect)
    topView = cv2.warpPerspective(frame, matrix, (warp_width, warp_height))
    
    # Show the result in a new windows
    cv2.imshow('TopView',topView)

In [11]:
# Draw the detected car region
def draw_car_regions():
    for(x, y, w, h) in cars:
        # Apply offset of ROI
        x += car_roi_x1
        y += car_roi_y1

        # Draw rect and text
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.rectangle(frame, (x, y-40), (x + w, y), (0, 0, 255), -2)
        cv2.putText(frame, 'Car', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

In [12]:
# Main
while True:
    ret, frame = cap.read()
    
    # loop the video
    if not ret:
        cap = cv2.VideoCapture(video)
        continue
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    roi_line = gray[line_roi_y1:line_roi_y2, line_roi_x1:line_roi_x2].copy() # ROI for detect lanes
    roi_car = gray[car_roi_y1:car_roi_y2, car_roi_x1:car_roi_x2].copy() # ROI for detect cars
    
    # Haar Cascade
    cars = car_haar_cascade()
    
    # Gaussian Blur
    roi_b = blur()
    
    # Canny Detection
    roi_c = canny()

    # Hough Transform
    lines = cv2.HoughLinesP(roi_c, rho = 1, theta = np.pi/60, threshold = 20, minLineLength = 200, maxLineGap = 150)
    
    # Draw ROI area
    draw_roi(roi_c)
    
    # Draw Hough Lines
    draw_lines()
        
    # Show perspective warp
    perspective_warp()

    # Draw the detected car region
    draw_car_regions()
    
    cv2.imshow('video',frame)
    
    key = cv2.waitKey(25)
    if(key == 27):
        break

cap.release()
cv2.destroyAllWindows()